# Imports

In [1]:
# import requests 
# import re 
# from bs4 import BeautifulSoup 
# import pandas as pd
# import regex as re
# import os
import imports as i
import importlib
importlib.reload(i)

c:\Users\Veronii\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<module 'imports' from 'c:\\Users\\Veronii\\Desktop\\data-wild-west\\code\\imports.py'>

# Functions for web-scraping

In [2]:
def check_response(url):
    response = i.requests.get(url)
    status_code = response.status_code
    if status_code != 200:
        raise Exception(f'Error {status_code} ({i.requests.status_codes._codes[status_code]})')
    else:
        html_text = response.content
        print(f'Request successful')
        return html_text

def make_soup(html_text):
    return i.BeautifulSoup(html_text, 'html.parser')

def get_contents(soup):
    entries = []
    keys = ['datetime', 'name', 'rating', 'title', 'review', 'event_time']
    reviews = soup.find_all('div', class_ = 'styles_reviewCardInner__EwDq2')
    
    for review in reviews:
        try:
            text = review.find('p', class_ = 'typography_body-l__KUYFJ typography_appearance-default__AAY17 typography_color-black__5LYEn').get_text()
        except AttributeError:
            continue
        
        date = (review.find('time')).attrs['datetime']
        name = review.find('span', class_ = 'typography_heading-xxs__QKBS8 typography_appearance-default__AAY17').get_text()
        rating = i.re.search('(\d+)(?=\s*.svg)', str(review.find('div', class_ =  'star-rating_starRating__4rrcf star-rating_medium__iN6Ty')))[0]
        title = review.find('h2', class_ = 'typography_heading-s__f7029 typography_appearance-default__AAY17').get_text()
        event = review.find('p', class_ = 'typography_body-m__xgxZ_ typography_appearance-default__AAY17').get_text()[20:]
        
        values = [date, name, rating, title, text, event]
        d = dict(zip(keys, values))
        entries.append(d)

    return entries

def make_df(df, d):
    df_new = i.pd.DataFrame.from_dict(d)
    return i.pd.concat([df, df_new], ignore_index=True, axis = 0) 

# Execute scrape. Change *n* to set desired number of reviews.

In [3]:
n = 100

df = i.pd.DataFrame()

for j in range(1,n):
    if len(df)>=n:
        print(f'Scrape complete! \n {j} review pages have been scanned, total reviews is {len(df)}.')
        break
    url = 'https://dk.trustpilot.com/review/puregym.dk?page={}&sort=recency'.format(j)
    html = check_response(url)
    soup = make_soup(html)
    d = get_contents(soup)
    df = make_df(df, d)

Request successful
Request successful
Request successful
Request successful
Request successful
Request successful
Scrape complete! 
 7 review pages have been scanned, total reviews is 107.


# Preview df

In [4]:
df.head()

,datetime,name,rating,title,review,event_time
0,2023-11-18T17:55:07.000Z,jan,5,PureGym Dannebrogsgade Aalborg,går i PureGym i Dannebrogsgade i Aalborg og ha...,16. november 2023
1,2023-11-17T21:30:59.000Z,Heidi Pernille Hviid,5,Først gives max antal stjerner…,Først max antal stjerner for personalet. Især ...,17. november 2023
2,2023-11-19T19:31:45.000Z,Anne,5,Jeg fik hjælp af en af instruktørerne…,Jeg fik hjælp af en af instruktørerne pga. en ...,19. november 2023
3,2023-11-19T20:07:05.000Z,Stefan,1,lang liste af fejl,Der er så mange fejl og mangler på Bryggervang...,15. november 2023
4,2023-11-18T11:51:42.000Z,Jolanta Hansen,5,Gode hold og holdtrænere,Gode hold og holdtrænere. Altid engagerede und...,18. november 2023


# Save the df as CSV in current working directory

In [5]:
cwd = i.os.getcwd()
path = cwd + "../data/raw_data/TrustPilot_reviews.csv"
df.to_csv(path, encoding='utf-16')